In [2]:
from typing import Dict
import itertools
import time

import torch
import tensorrt as trt

import numpy as np

from transformers import XLMRobertaTokenizerFast
from handlers.full.extra_files.model import ModelWithGlobalProjection, ModelOutputWithProjection, Model

from tensorrt_inference.backend import load_engine

In [3]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

In [4]:
model_path = "/home/g.racic/sync/pca-targetting/model.pt"
model = ModelWithGlobalProjection(
    model=Model(233), projection=torch.nn.Linear(768, 100, bias=False)
)
model.load_state_dict(torch.load(model_path))
model.eval()

ModelWithGlobalProjection(
  (model): Model(
    (base_model): XLMRobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_featur

In [5]:
def to_model_output_with_projection(trt_outputs):
    return ModelOutputWithProjection(
        text_embedding=torch.Tensor(trt_outputs[0]), proj_text_embedding=torch.Tensor(trt_outputs[1])
    )

In [13]:
def preprocess(text):
    return tokenizer(
            [text],
            max_length=96,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
    )

In [37]:
def inference(inputs, model_fn):
    with torch.no_grad():
        return model_fn(inputs)

In [8]:
torch.set_num_interop_threads(1)
torch.set_num_threads(1)

## Vanilla pytorch on CPU

In [38]:
def inference_fn(model, device):
    def _inference_fn(inputs: Dict[str, torch.Tensor]):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        model_output = model(**inputs)
        torch.cuda.synchronize()
        return model_output

    return _inference_fn

In [39]:
def benchmark(model_fn, input_data, batch_size, nwarmup=50, nruns=1000):
    _data = itertools.cycle(input_data)
    print("Warm up ...")
    with torch.no_grad():
        for n in range(nwarmup):
            inference(preprocess(next(_data)), model_fn)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            inference(preprocess(next(_data)), model_fn)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, avg batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))
 
    print('Average throughput: %.2f example/second'%(batch_size/np.mean(timings)))

In [11]:
text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque tincidunt libero at urna accumsan, ut fermentum lacus placerat. Nulla elementum ut orci in tristique. Sed efficitur non sem ut vulputate. Duis risus eros, convallis sit amet maximus sed, molestie in lacus. In hac habitasse platea dictumst. Phasellus vel fermentum orci, nec porttitor ante. Proin malesuada odio a velit ultrices, at consectetur ante posuere.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
Aenean faucibus purus in velit malesuada convallis. Aliquam tristique, mi in hendrerit aliquam, diam nisi finibus dui, non faucibus ligula lacus eget magna. Sed laoreet ex ante, sit amet placerat risus blandit nec. Donec interdum arcu enim, quis elementum erat convallis in. Curabitur a interdum tortor. Praesent id consequat urna. Sed ornare pellentesque velit, in facilisis quam. Curabitur fringilla vitae quam in lacinia. Vivamus ut consequat leo. Nunc vulputate, lorem a ultrices egestas, eros mi accumsan felis, in sollicitudin mi turpis vel tellus. Cras tincidunt, lectus ac euismod eleifend, nunc ipsum gravida nisl, ac gravida lacus magna non dui. Pellentesque rhoncus ullamcorper convallis.
"""

In [17]:
benchmark(inference_fn(model, "cpu"), text, 1, nwarmup=50, nruns=50)

Warm up ...
Start timing ...
Iteration 10/50, avg batch time 159.22 ms
Iteration 20/50, avg batch time 159.28 ms
Iteration 30/50, avg batch time 159.24 ms
Iteration 40/50, avg batch time 159.35 ms
Iteration 50/50, avg batch time 159.34 ms
Average throughput: 6.28 example/second


## Int8 XLM-Roberta-Base on CPU

In [18]:
int8_cpu_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear, torch.nn.Embedding}, dtype=torch.qint8
)
int8_cpu_model = int8_cpu_model.eval()

In [19]:
benchmark(inference_fn(int8_cpu_model, "cpu"), text, 1, nwarmup=50, nruns=50)

Warm up ...
Start timing ...
Iteration 10/50, avg batch time 95.84 ms
Iteration 20/50, avg batch time 95.58 ms
Iteration 30/50, avg batch time 95.45 ms
Iteration 40/50, avg batch time 95.44 ms
Iteration 50/50, avg batch time 95.32 ms
Average throughput: 10.49 example/second


## Vanilla XLM-Robera-Base on GPU

In [20]:
model = model.to("cuda:0")

In [50]:
benchmark(inference_fn(model, "cuda:0"), text, 1, nwarmup=50, nruns=50)

Warm up ...
Start timing ...
Iteration 10/50, avg batch time 8.06 ms
Iteration 20/50, avg batch time 8.06 ms
Iteration 30/50, avg batch time 8.05 ms
Iteration 40/50, avg batch time 8.04 ms
Iteration 50/50, avg batch time 8.02 ms
Average throughput: 124.64 example/second


## Fp16/Int8 TRT XLM-Roberta-Base on GPU

In [63]:
def inference_fn_trt(model, device):
    def _inference_fn(inputs: Dict[str, torch.Tensor]):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        model_output = model(inputs)
        torch.cuda.synchronize()
        return model_output

    return _inference_fn

In [23]:
trt_model_path = "/home/g.racic/mode_trt"

In [24]:
trt_logger = trt.Logger(trt.Logger.VERBOSE)
runtime = trt.Runtime(trt_logger)

In [25]:
trt_model = load_engine(
    runtime=runtime, engine_file_path=trt_model_path
)

In [64]:
benchmark(inference_fn_trt(trt_model, "cpu"), text, 1, nwarmup=50, nruns=50)

Warm up ...
Start timing ...
Iteration 10/50, avg batch time 2.25 ms
Iteration 20/50, avg batch time 2.21 ms
Iteration 30/50, avg batch time 2.18 ms
Iteration 40/50, avg batch time 2.17 ms
Iteration 50/50, avg batch time 2.16 ms
Average throughput: 462.62 example/second
